In [1]:
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import roc_auc_score

In [2]:
%reload_ext watermark
%watermark -a "Pedro Marcello"

Author: Pedro Marcello



In [3]:
df = pd.read_csv("Combined Data.csv")

In [4]:
df.shape

(53043, 3)

In [5]:
df.head(10)

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
5,5,"every break, you must be nervous, like somethi...",Anxiety
6,6,"I feel scared, anxious, what can I do? And may...",Anxiety
7,7,Have you ever felt nervous but didn't know why?,Anxiety
8,8,"I haven't slept well for 2 days, it's like I'm...",Anxiety
9,9,"I'm really worried, I want to cry.",Anxiety


In [6]:
pd.set_option('display.max_colwidth', 130)  

In [7]:
df.sample(10)

,Unnamed: 0,statement,status
34002,34002,"Worried about smoking damage So I’m 18, I’ve been smoking from when I was 13 (one a day) going up to 3/4 a day for a while whe...",Anxiety
49656,49656,I have a Master’s degree and I just want to stock shelves I’m coming to terms with the fact that I’m not a good fit for my fie...,Bipolar
42362,42362,steve buscemi shut up you i am hungry and in britain home of the most boring suckassy breakfast in the world i miss dennys,Normal
13407,13407,It only gets worse and worse and the pain keeps growing. I literally thought weed was either helping or harming but it does no...,Suicidal
18454,18454,Does anyone know a quick way to kill myself? The fastest way to die?,Suicidal
32450,32450,really? who with?,Normal
23226,23226,"I had a major breakdown tonight and was scared of being alone. I knew I would not do anything drastic, but it did not matter. ...",Depression
50692,50692,this disorder has stopped me from pushing myself for the better me. Was scrolling on tiktok when I saw my previous school mate...,Personality disorder
12750,12750,"So I am 19 (male) and am having a really shit day today (even for me), I work as a carer and my favourite client died today an...",Depression
33869,33869,i'll use a damp cloth.,Normal


In [8]:
df.status.value_counts(normalize= True)

status
Normal                  0.308259
Depression              0.290406
Suicidal                0.200837
Anxiety                 0.073299
Bipolar                 0.054239
Stress                  0.050318
Personality disorder    0.022642
Name: proportion, dtype: float64

In [9]:
df = df.drop(columns= ['Unnamed: 0'])

In [10]:
df.shape

(53043, 2)

In [11]:
df = df[~df['status'].isin(['Bipolar', 'Stress', 'Personality disorder'])]

In [12]:
df.status.value_counts(normalize= True)

status
Normal        0.353184
Depression    0.332729
Suicidal      0.230106
Anxiety       0.083981
Name: proportion, dtype: float64

In [13]:
df.count()

statement    46240
status       46296
dtype: int64

In [14]:
df = df.dropna()

In [15]:
df.count()

statement    46240
status       46240
dtype: int64

In [16]:
df.loc[:, 'status'] = df['status'].replace({
    'Normal': 1,
    'Depression': 2,
    'Suicidal': 3,
    'Anxiety': 4
})

C:\Users\pedro\AppData\Local\Temp\ipykernel_8700\4255388099.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'status'] = df['status'].replace({


In [17]:
df.head()

,statement,status
0,oh my gosh,4
1,"trouble sleeping, confused mind, restless heart. All out of tune",4
2,"All wrong, back off dear, forward doubt. Stay in a restless and restless place",4
3,I've shifted my focus to something else but I'm still worried,4
4,"I'm restless and restless, it's been a month now, boy. What do you mean?",4


In [18]:
def remover_caracter_especial(texto):
    rem = ''
    for i in texto:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '  
    return rem

In [19]:
def converter_minusculo(texto):
    return texto.lower()

In [20]:
def remover_stopwords(texto):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(str(texto))
    return [w for w in words if w not in stop_words]

In [21]:
def stemmer(texto):
    objeto_stemmer = SnowballStemmer('english')
    return " ".join([objeto_stemmer.stem(w) for w in texto])

In [22]:
def processar_texto(texto):
    texto = remover_caracter_especial(texto) 
    texto = converter_minusculo(texto)        
    texto = remover_stopwords(texto)          
    texto = stemmer(texto)                    
    return texto

In [23]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
%%time
df.statement = df.statement.apply(processar_texto)

CPU times: total: 49.6 s
Wall time: 50.6 s


In [25]:
df.sample(15)

,statement,status
31187,eight,1
26229,know subreddit allow crosspost copi past post chronic fatigu sub even though post fatigu also suffer bipolar depress thought m...,2
5113,let open bismillah,1
18223,hate respons want kill meh,2
52242,took ashwagandha experienc horribl pain anyon els go 22ftm start ashwagandha 300mg gummi rough 1 everi 12 hour week half ago c...,4
12295,last month everyth touch turn shit girlfriend two year left anoth guy car keep break saw ex today told still love could make w...,3
34349,lump breast help day ago discov firm oval shape lump side right breast smooth kind press firm feel pain press kind feel like b...,4
45232,someon somewher like sheepish http www smartestgam com gotdhistori php id,1
3189,yes know fat also choos want skinni ning yaono ancen e fat long fast eat freeli,1
25054,want kms usual studi final l0l,3


In [26]:
x = np.array(df.statement.values)

In [27]:
y = np.array(df.status.values, dtype=int)

In [28]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [29]:
vetorizador = CountVectorizer(max_features= 1000)

In [30]:
x_treino_f = vetorizador.fit_transform(x_treino).toarray()

In [31]:
x_teste_f = vetorizador.transform(x_teste).toarray()

In [32]:
print("x_treino_f:", x_treino_f.shape)
print("y_treino:", y_treino.shape)

x_treino_f: (36992, 1000)
y_treino: (36992,)


In [33]:
print("x_teste_f:", x_teste_f.shape)
print("y_teste:", y_teste.shape)

x_teste_f: (9248, 1000)
y_teste: (9248,)


### Modelo v1 - GaussianNB

In [35]:
print(type(y_treino))  # Deve mostrar <class 'numpy.ndarray'>
print(y_treino.dtype)  # Verifique o tipo de dados interno
print(np.unique(y_treino))  # Verifique os valores únicos em y_treino


<class 'numpy.ndarray'>
int32
[1 2 3 4]


In [36]:
modelo_v1 = GaussianNB()

In [37]:
modelo_v1.fit(x_treino_f, y_treino)

GaussianNB()

In [38]:
y_proba = modelo_v1.predict_proba(x_teste_f)
auc = roc_auc_score(y_teste, y_proba, multi_class='ovr')
print("AUC do Modelo GaussianNB =", auc)

AUC do Modelo GaussianNB = 0.8303894970749225


### Modelo v2 - BernoulliNB

In [40]:
modelo_v2 = BernoulliNB()

In [41]:
modelo_v2.fit(x_treino_f, y_treino)

BernoulliNB()

In [42]:
y_proba = modelo_v2.predict_proba(x_teste_f)
auc = roc_auc_score(y_teste, y_proba, multi_class='ovr')
print("AUC do Modelo GaussianNB =", auc)

AUC do Modelo GaussianNB = 0.8553880761667833


In [43]:
%reload_ext watermark
%watermark -a "Pedro Marcello"

Author: Pedro Marcello



In [44]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.27.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 3, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [45]:
%watermark --iversions

numpy  : 1.26.4
nltk   : 3.9.1
sklearn: 1.5.1
json   : 2.0.9
pandas : 2.2.2

